In [1]:
#basic package
import json
import tqdm
import os
import numpy as np
import glob
import pandas as pd
import sys
import shutil
import time
import datetime as dt
import itertools
import time
import re
from collections import Counter
from collections import defaultdict
import operator
import math

#plot
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
sys.path.append('C:\\Users\\camil\\Desktop\\animals_code\\entropy')
from entropy.entropy import sample_entropy

In [3]:
from UTILS import time_series_henColumn_tsRow, ZoneVariable, HenVariable, DistributionEntropy, \
heatmap_duration_perzone_perhen, TimeSeriesPlot, entropy_compare_session
import config_mobility as config

we restrict the time series to one value per 60 seconds 
we compute the complexity variables each 30 minutes 
each variables includes the values of at least the last 120.00 minutes (i.e. are using 120.00 values)


In [4]:
path_extracted_data = config.path_extracted_data
id_run = config.id_run
nbr_sec = config.nbr_sec

# Session Variable Verification

In [12]:
#choose teh session to look at
session = '4B'
#level
ts_name = 'time_serie_day' 

In [13]:
#download the data
df = pd.read_csv(os.path.join(path_extracted_data, id_run+'_'+ts_name+'_'+session+'_variables.csv'), sep=';',
                parse_dates = ['level','FirstTimestamp_zone_1','FirstTimestamp_zone_2', 'FirstTimestamp_zone_3', 
                               'FirstTimestamp_zone_4', 'FirstTimestamp_zone_5'])
print(df.shape)
df.head(3)

(354, 44)


HenID      level  duration_zone_1  duration_zone_2  duration_zone_3  \
0  hen_100 2017-01-13           6738.0          23373.0          21785.0   
1  hen_100 2017-01-14           5388.0          24344.0          20834.0   
2  hen_100 2017-01-15           4645.0          23023.0          23245.0   

   duration_zone_4  duration_zone_5  verification_daily_total_duration  \
0           2104.0              0.0                            54000.0   
1           3396.0             38.0                            54000.0   
2           2583.0            504.0                            54000.0   

                                  dur_values  \
0    [6738.0, 23373.0, 21785.0, 2104.0, 0.0]   
1   [5388.0, 24344.0, 20834.0, 3396.0, 38.0]   
2  [4645.0, 23023.0, 23245.0, 2583.0, 504.0]   

                               dur_values_normalized  \
0  [0.12477777777777778, 0.43283333333333335, 0.4...   
1  [0.09977777777777778, 0.45081481481481483, 0.3...   
2  [0.08601851851851852, 0.42635185185185187, 0.4...   

                         ...                         Total_number_transition  \
0                        ...                                              84   
1                        ...                                              83   
2                        ...                                              68   

  distribution_entropy nbr_bouts_zone_3 nbr_bouts_zone_4 nbr_bouts_zone_2  \
0             1.114801               27                3               39   
1             1.135665               30                4               38   
2             1.126356               23                4               30   

   nbr_bouts_zone_1  nbr_bouts_zone_5  Max_duration_zone_4  SampEnt_order2  \
0                16                 0                 2044        0.002979   
1                11                 1                 2119        0.003041   
2                11                 1                 1496        0.002341   

                                     SampEnt_perZone  
0  {'zone_1': 0.04557403144807681, 'zone_2': 0.15...  
1  {'zone_1': 0.02690476458642156, 'zone_2': 0.15...  
2  {'zone_1': 0.026105446031409685, 'zone_2': 0.1...  

[3 rows x 44 columns]

## verify value in each variable

In [16]:
df[(df['HenID']=='hen_103')&(df['level']=='2017-01-18')][['list_of_durations', 'dico_zone_duration', 'FirstTimestamp_zone_1', 
                                                          'FirstTimestamp_zone_2', 'FirstTimestamp_zone_3', 
                                                          'FirstTimestamp_zone_4', 'FirstTimestamp_zone_5']]

list_of_durations  \
23  [269, 4191, 43, 2, 6, 150, 812, 3098, 521, 366...   

                                   dico_zone_duration FirstTimestamp_zone_1  \
23  {'zone_4': [2, 4, 18, 4191], 'zone_3': [6, 20,...                   NaT   

   FirstTimestamp_zone_2 FirstTimestamp_zone_3 FirstTimestamp_zone_4  \
23   2017-01-18 03:15:11   2017-01-18 02:00:00   2017-01-18 02:04:29   

   FirstTimestamp_zone_5  
23                   NaT

In [21]:
df[(df['HenID']=='hen_103')&(df['level']=='2017-01-18')]['list_of_durations'].values[0]

'[269, 4191, 43, 2, 6, 150, 812, 3098, 521, 3665, 1579, 1032, 1093, 4080, 1265, 1503, 150, 2549, 1320, 2122, 1079, 1203, 366, 1266, 372, 232, 55, 470, 308, 335, 76, 18, 20, 1287, 278, 1469, 372, 1769, 581, 708, 493, 1741, 302, 2273, 437, 1011, 3337, 287, 901, 514, 708, 4, 35, 243]'

In [ ]:
150+3665+3098+1032+4080+150

In [36]:
df.columns

Index(['HenID', 'level', 'duration_zone_1', 'duration_zone_2',
       'duration_zone_3', 'duration_zone_4', 'duration_zone_5',
       'verification_daily_total_duration', 'dur_values',
       'dur_values_normalized', 'FirstTimestamp_zone_1',
       'FirstTimestamp_zone_2', 'FirstTimestamp_zone_3',
       'FirstTimestamp_zone_4', 'FirstTimestamp_zone_5', 'Total_number_zone',
       'RunSampEnt_onLastTsOfEachLevel', 'RunDistEnt_onLastTsOfEachLevel',
       'RunEnt_onLastTsOfEachLevel_nbr_value',
       'RunEnt_onLastTsOfEachLevel_ts_value',
       'RunSampEnt_onLastTsOfEachLevel_1', 'RunSampEnt_onLastTsOfEachLevel_2',
       'RunSampEnt_onLastTsOfEachLevel_3', 'RunSampEnt_onLastTsOfEachLevel_4',
       'RunSampEnt_onLastTsOfEachLevel_5', 'list_of_durations',
       'Max_duration_zones', 'Max_duration', 'Min_duration', 'Median_duration',
       'Average_duration', 'Variance_duration', 'dico_zone_duration',
       'Total_number_transition', 'distribution_entropy', 'SampEnt_order2',
       

#### basic columns

In [37]:
# all possible level
#level is not zones, but its the level of the analysis usually either day or night, but for mobility we only use day (i.e. 
#ts_name = 'time_serie_day' )
df['level'].unique()

array(['2016-11-17T00:00:00.000000000', '2016-11-18T00:00:00.000000000',
       '2016-11-19T00:00:00.000000000', '2016-11-20T00:00:00.000000000',
       '2016-11-21T00:00:00.000000000', '2016-11-22T00:00:00.000000000'],
      dtype='datetime64[ns]')

In [38]:
#number of level per hen
df_ = df.groupby(['HenID'])['level'].agg(lambda x: tuple(x)).reset_index()
df_['nbr_level'] = df_['level'].map(lambda x: len(x))
df_.head(5)

HenID                                              level  nbr_level
0   hen_1  (2016-11-17 00:00:00, 2016-11-18 00:00:00, 201...          6
1  hen_10  (2016-11-17 00:00:00, 2016-11-18 00:00:00, 201...          6
2  hen_12  (2016-11-17 00:00:00, 2016-11-18 00:00:00, 201...          6
3  hen_13  (2016-11-17 00:00:00, 2016-11-18 00:00:00, 201...          6
4  hen_14  (2016-11-17 00:00:00, 2016-11-18 00:00:00, 201...          6

In [39]:
#number of hours per level in seconds
df['verification_daily_total_duration'].unique()

array([54000.])

##### duration

In [11]:
row = 1
print(df.iloc[row]['dico_zone_duration'])
display(df[['duration_zone_1', 'duration_zone_2','duration_zone_3', 'duration_zone_4', 'duration_zone_5']][row:row+1])
df[['Max_duration_zones','Max_duration', 'Min_duration', 'Median_duration', 'Average_duration',
    'Variance_duration','Max_duration_zone_4']][row:row+1]

{'zone_5': [1, 2, 3, 150, 151, 175, 490, 932, 1196, 3406, 3653, 3676, 4338, 4359, 5827, 5920, 9252], 'zone_3': [11, 15, 44, 561], 'zone_2': [23], 'zone_4': [25, 26, 38, 42, 51, 104, 142, 155, 167, 260, 271, 294, 348, 551, 567, 647, 761, 1099, 4267]}


duration_zone_1  duration_zone_2  duration_zone_3  duration_zone_4  \
1              0.0             23.0            631.0           9815.0   

   duration_zone_5  
1          43531.0

Max_duration_zones  Max_duration  Min_duration  Median_duration  \
1             zone_5          9252             1            271.0   

   Average_duration  Variance_duration  Max_duration_zone_4  
1       1317.073171       4.491813e+06                 4267

##### ordered list of duration per zone FOR chi2 distance computation

In [41]:
#ordered list of duration per zone for chi2 distance computation
#Note that its not necessarily the intuitive zone order
row1 = 0 ; row2 = 1
display(df[['duration_zone_1', 'duration_zone_2','duration_zone_3', 'duration_zone_4', 'duration_zone_5','dur_values', 
            'dur_values_normalized']][row1:row1+1])
df[['duration_zone_1', 'duration_zone_2','duration_zone_3', 'duration_zone_4', 'duration_zone_5','dur_values', 
    'dur_values_normalized']][row2:row2+1]

duration_zone_1  duration_zone_2  duration_zone_3  duration_zone_4  \
0           7229.0          27494.0           9959.0           2756.0   

   duration_zone_5                                 dur_values  \
0           6562.0  [7229.0, 27494.0, 9959.0, 2756.0, 6562.0]   

                               dur_values_normalized  
0  [0.13387037037037036, 0.5091481481481481, 0.18...

duration_zone_1  duration_zone_2  duration_zone_3  duration_zone_4  \
1           2433.0          29321.0          11824.0           3820.0   

   duration_zone_5                                  dur_values  \
1           6602.0  [2433.0, 29321.0, 11824.0, 3820.0, 6602.0]   

                               dur_values_normalized  
1  [0.04505555555555556, 0.5429814814814815, 0.21...

In [42]:
sum(eval(df['dur_values_normalized'].iloc[row1])), sum(eval(df['dur_values_normalized'].iloc[row2]))

(0.9999999999999999, 1.0)

##### nbr transition and bouts

In [43]:
row=0
print(df.iloc[row]['dico_zone_duration'])
print(len([i for x in eval(df.iloc[row]['dico_zone_duration']).values() for i in x]))
df[['Total_number_transition','nbr_bouts_zone_5', 'nbr_bouts_zone_4', 'nbr_bouts_zone_3',
'nbr_bouts_zone_2', 'nbr_bouts_zone_1','Total_number_zone']].iloc[row: row+1]

{'zone_3': [3, 5, 9, 12, 16, 44, 47, 56, 68, 77, 81, 135, 140, 142, 149, 180, 180, 186, 188, 200, 220, 228, 250, 250, 262, 272, 273, 312, 312, 377, 404, 424, 428, 440, 472, 492, 501, 540, 546, 1038], 'zone_4': [7, 8, 9, 13, 14, 15, 16, 18, 22, 24, 24, 27, 37, 38, 44, 53, 56, 67, 74, 82, 97, 108, 148, 148, 152, 166, 404, 885], 'zone_5': [9, 16, 34, 39, 53, 62, 81, 85, 90, 99, 105, 138, 138, 145, 268, 503, 662, 699, 869, 966, 1501], 'zone_2': [16, 16, 20, 21, 23, 32, 41, 48, 56, 56, 66, 88, 91, 93, 132, 139, 147, 176, 194, 198, 204, 237, 253, 293, 361, 372, 374, 383, 463, 499, 554, 556, 651, 764, 774, 781, 807, 835, 906, 948, 952, 952, 1025, 1080, 1114, 1330, 1345, 1663, 1847, 3518], 'zone_1': [80, 121, 226, 237, 296, 379, 438, 469, 746, 909, 1080, 1122, 1126]}
152


Total_number_transition  nbr_bouts_zone_5  nbr_bouts_zone_4  \
0                      151                21                28   

   nbr_bouts_zone_3  nbr_bouts_zone_2  nbr_bouts_zone_1  Total_number_zone  
0                40                50                13                  5

##### complexity variables

In [44]:
df[['dur_values','dur_values_normalized','distribution_entropy', 'SampEnt_order2', 'SampEnt_perZone']].head(3)

dur_values  \
0   [7229.0, 27494.0, 9959.0, 2756.0, 6562.0]   
1  [2433.0, 29321.0, 11824.0, 3820.0, 6602.0]   
2  [2281.0, 30201.0, 10644.0, 3014.0, 7860.0]   

                               dur_values_normalized  distribution_entropy  \
0  [0.13387037037037036, 0.5091481481481481, 0.18...              1.332623   
1  [0.04505555555555556, 0.5429814814814815, 0.21...              1.248141   
2  [0.04224074074074074, 0.5592777777777778, 0.19...              1.220353   

   SampEnt_order2                                    SampEnt_perZone  
0        0.004340  {'zone_1': 0.0006268755926943874, 'zone_2': 0....  
1        0.003862  {'zone_1': 0.00032417862162414707, 'zone_2': 0...  
2        0.003539  {'zone_1': 0.00040294560333948854, 'zone_2': 0...

In [45]:
#lets verify distribution entropy, by computing it on the duration values per zone, instead of the ts_zone list
row = 0
print(df.iloc[row]['distribution_entropy'])
dn = eval(df.iloc[row]['dur_values_normalized'])
d = eval(df.iloc[row]['dur_values'])
from scipy.stats import entropy
print(entropy(dn), entropy(d))

1.332623136755784
1.3326231367557841 1.3326231367557841


In [46]:
#we dont have the initial data to verify variables 'SampEnt_order2', 'SampEnt_perZone', but lets see simply the value for now
row = 0
df.iloc[row]['SampEnt_perZone']

"{'zone_1': 0.0006268755926943874, 'zone_2': 0.0037095757801931272, 'zone_3': 0.0021195087900572384, 'zone_4': 0.0011479262453591443, 'zone_5': 0.000941755289063495}"

#### running complexity variables

In [47]:
#for now lets simply look at their values
df[['level','RunEnt_onLastTsOfEachLevel_nbr_value','RunEnt_onLastTsOfEachLevel_ts_value',
    'RunSampEnt_onLastTsOfEachLevel', 'RunDistEnt_onLastTsOfEachLevel','RunSampEnt_onLastTsOfEachLevel_1', 
    'RunSampEnt_onLastTsOfEachLevel_2','RunSampEnt_onLastTsOfEachLevel_3', 'RunSampEnt_onLastTsOfEachLevel_4',
    'RunSampEnt_onLastTsOfEachLevel_5']].head(5)

level  RunEnt_onLastTsOfEachLevel_nbr_value  \
0 2016-11-17                                   900   
1 2016-11-18                                  1800   
2 2016-11-19                                  2700   
3 2016-11-20                                  3600   
4 2016-11-21                                  4500   

  RunEnt_onLastTsOfEachLevel_ts_value  RunSampEnt_onLastTsOfEachLevel  \
0                 2016-11-17 16:59:59                        0.206040   
1                 2016-11-18 16:59:59                        0.206501   
2                 2016-11-19 16:59:59                        0.196560   
3                 2016-11-20 16:59:59                        0.199960   
4                 2016-11-21 16:59:59                        0.192006   

   RunDistEnt_onLastTsOfEachLevel  RunSampEnt_onLastTsOfEachLevel_1  \
0                        1.331652                          0.037273   
1                        1.305574                          0.027557   
2                        1.281446                          0.026318   
3                        1.270405                          0.027016   
4                        1.263202                          0.025992   

   RunSampEnt_onLastTsOfEachLevel_2  RunSampEnt_onLastTsOfEachLevel_3  \
0                          0.195022                          0.101396   
1                          0.192460                          0.111823   
2                          0.187272                          0.108082   
3                          0.189185                          0.111150   
4                          0.182547                          0.107046   

   RunSampEnt_onLastTsOfEachLevel_4  RunSampEnt_onLastTsOfEachLevel_5  
0                          0.027244                          0.036312  
1                          0.030198                          0.036391  
2                          0.027408                          0.034064  
3                          0.028733                          0.031398  
4                          0.028050                          0.031173

## match some hen movement from the initial file "movements_timeseries" to variables values

In [48]:
#note that the "list_of_durations" is ordered by when it occured in the time, while the dico_zone_duration is ordered by 
#ascending value
HenID = 'hen_1'
df[df['HenID']==HenID][['level','list_of_durations','dico_zone_duration','FirstTimestamp_zone_1',
       'FirstTimestamp_zone_2', 'FirstTimestamp_zone_3','FirstTimestamp_zone_4', 'FirstTimestamp_zone_5']]

level                                  list_of_durations  \
0 2016-11-17  [138, 14, 1345, 428, 361, 250, 952, 16, 13, 26...   
1 2016-11-18  [85, 11, 1548, 451, 1170, 33, 39, 426, 186, 13...   
2 2016-11-19  [145, 21, 1277, 619, 1064, 16, 49, 833, 35, 41...   
3 2016-11-20  [67, 197, 454, 495, 92, 1349, 169, 156, 1527, ...   
4 2016-11-21  [54, 11, 39, 355, 1183, 242, 144, 2854, 173, 6...   
5 2016-11-22  [1029, 67, 1318, 317, 654, 402, 22, 217, 140, ...   

                                  dico_zone_duration FirstTimestamp_zone_1  \
0  {'zone_3': [3, 5, 9, 12, 16, 44, 47, 56, 68, 7...   2016-11-17 09:59:14   
1  {'zone_4': [1, 1, 2, 3, 7, 8, 11, 12, 13, 14, ...   2016-11-18 09:58:48   
2  {'zone_4': [1, 1, 7, 9, 9, 21, 25, 33, 35, 37,...   2016-11-19 10:56:16   
3  {'zone_3': [1, 2, 3, 4, 11, 19, 39, 48, 59, 70...   2016-11-20 09:58:56   
4  {'zone_5': [1, 1, 3, 11, 37, 45, 62, 63, 76, 1...   2016-11-21 10:04:30   
5  {'zone_4': [2, 4, 9, 15, 17, 22, 27, 40, 45, 6...   2016-11-22 09:58:57   

  FirstTimestamp_zone_2 FirstTimestamp_zone_3 FirstTimestamp_zone_4  \
0   2016-11-17 02:02:32   2016-11-17 02:24:57   2016-11-17 02:02:18   
1   2016-11-18 02:01:36   2016-11-18 02:27:24   2016-11-18 02:01:25   
2   2016-11-19 02:02:46   2016-11-19 02:24:03   2016-11-19 02:02:25   
3   2016-11-20 02:01:07   2016-11-20 02:04:24   2016-11-20 02:47:03   
4   2016-11-21 02:01:05   2016-11-21 02:01:44   2016-11-21 02:00:00   
5   2016-11-22 02:18:16   2016-11-22 02:40:14   2016-11-22 02:17:09   

  FirstTimestamp_zone_5  
0   2016-11-17 02:00:00  
1   2016-11-18 02:00:00  
2   2016-11-19 02:00:00  
3   2016-11-20 02:00:00  
4   2016-11-21 02:00:54  
5   2016-11-22 02:00:00

# Event Variable Verification

In [ ]:
#TODO